In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Display the Columns and Data and Count
purchase_data.columns
purchase_data.head()
purchase_data.count()
purchase_data.dtypes


Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [2]:
#Displaying the Total Number of Players
purchase_data.columns
player_count = len(purchase_data["SN"].unique())
summary = pd.DataFrame({"Total Player":[player_count]})
summary

,Total Player
0,576


In [4]:
# Run basic calculations to obtain number of unique items, average price, Number of Purchases, Total Revenue
item_count = len(purchase_data["Item Name"].unique())
average_price = "${:.2f}".format(purchase_data["Price"].mean())
number_of_purchases = purchase_data["Purchase ID"].count()
total_revenue = "${:.2f}".format(purchase_data["Price"].sum())

# Create a summary data frame to hold the results
summary = pd.DataFrame({"Number of Unique Items":[item_count],
                       "Average Price":[average_price],
                       "Number of Purchases":[number_of_purchases],
                       "Total Revenue": total_revenue})
summary           

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [6]:
#Create DataFrame of Unique Players
unique_data = purchase_data.filter(["SN","Gender", "Age"], axis=1)
unique_data = unique_data.sort_values("SN", ascending=False) 
unique_data.drop_duplicates(subset ="SN", keep = "first", inplace = True)

#Create Summary
gender_summary = unique_data[["SN","Gender"]].groupby("Gender").count()
gender_summary = gender_summary.rename(columns={"SN":"Total Count"})
gender_summary["Percentage of Players"] = gender_summary["Total Count"]*100/player_count
gender_summary["Percentage of Players"] = gender_summary["Percentage of Players"].map("{:.2f}%".format)


gender_summary


,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [7]:
#Creating Purchase Count Dataframe and Average Purchase Price Dataframe
gender_price_summary = purchase_data[["SN","Gender","Price"]].groupby("Gender").agg({
    "SN":"count",
    "Price":"mean"
})
gender_price_summary["Price"] = gender_price_summary["Price"].map("${:.2f}".format)
gender_price_summary = gender_price_summary.rename(columns={"SN":"Purchase Count",
                                                     "Price":"Average Purchase Price"})


#Create Total Purchase Value Dataframe
total_value = purchase_data[["Gender","Price"]].groupby(["Gender"]).sum()
total_value["Price"] = total_value["Price"].map("${:.2f}".format)
total_value = total_value.rename(columns={"Price":"Total Purchase"})

#Create Avg Total Purchase per Person Dataframe
avg_total = purchase_data[["SN","Gender","Price"]].groupby(["Gender","SN"]).sum()
avg_total = avg_total.groupby("Gender").mean()
avg_total["Price"] = avg_total["Price"].map("${:.2f}".format)
avg_total = avg_total.rename(columns={"Price":"Avg Total Purchase per Person"})
avg_total


#Create Summary Dataframe
gender_price_summary = pd.merge(gender_price_summary, total_value, on="Gender", how="outer")
gender_price_summary = pd.merge(gender_price_summary, avg_total, on="Gender", how="outer")
gender_price_summary

,Purchase Count,Average Purchase Price,Total Purchase,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
#Create DataFrame of Unique Players and Ages
#Display Age Demographics Dataframe
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_groups = ["<10", "11-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_summary = unique_data.loc[:,["SN","Age"]]
age_summary["Age Group"] = pd.cut(age_summary["Age"], bins, labels=age_groups)
age_summary = age_summary[["SN","Age Group"]].groupby("Age Group").count()
age_summary = age_summary.rename(columns={"SN":"Total Count"})
age_summary["Percentage of Players"] = age_summary["Total Count"]*100/player_count
age_summary["Percentage of Players"] = age_summary["Percentage of Players"].map("{:.2f}%".format)

age_summary



,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
11-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [9]:
#Creating Purchase Count Dataframe and Average Purchase Price Dataframe
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_groups = ["<10", "11-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_price_summary = purchase_data.loc[:,["SN","Age","Price"]]
age_price_summary["Age Group"] = pd.cut(age_price_summary["Age"], bins, labels=age_groups)
age_price_summary = age_price_summary.groupby("Age Group").agg({
    "SN":"count",
    "Price":"mean"
})
age_price_summary["Price"] = age_price_summary["Price"].map("${:.2f}".format)
age_price_summary = age_price_summary.rename(columns={"SN":"Purchase Count",
                                                     "Price":"Average Purchase Price"})

#Create Total Purchase Value Dataframe
total_value = purchase_data.loc[:,["Age","Price"]]
total_value["Age Group"] = pd.cut(total_value["Age"], bins, labels=age_groups)
total_value = total_value[["Age Group","Price"]].groupby(["Age Group"]).sum()
total_value["Price"] = total_value["Price"].map("${:.2f}".format)
total_value = total_value.rename(columns={"Price":"Total Purchase"})

#Create Avg Total Purchase per Person Dataframe
avg_total = purchase_data.loc[:,["SN","Age","Price"]]
avg_total["Age Group"] = pd.cut(avg_total["Age"], bins, labels=age_groups)
avg_total = avg_total[["SN","Age Group","Price"]].groupby(["SN","Age Group"]).sum()
avg_total = avg_total.groupby("Age Group").mean()
avg_total["Price"] = avg_total["Price"].map("${:.2f}".format)
avg_total = avg_total.rename(columns={"Price":"Avg Total Purchase per Person"})
avg_total


#Create Summary Dataframe
age_price_summary = pd.merge(age_price_summary, total_value, on="Age Group", how="outer")
age_price_summary = pd.merge(age_price_summary, avg_total, on="Age Group", how="outer")

age_price_summary

,Purchase Count,Average Purchase Price,Total Purchase,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
11-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [10]:
#Calculating the Top 5 Spenders' Purchase Count, Average Purchase Price, Total Purchase Value
#Creating A DataFrame Summary
purchase_data["Price2"] = purchase_data["Price"]
top5_spender_summary = purchase_data[["SN", "Price2", "Price", "Age"]].groupby("SN").agg({
    "Price":"sum",
    "Age":"count",
    "Price2":"mean"
})
top5_spender_summary = top5_spender_summary.sort_values("Price", ascending=False)
top5_spender_summary = top5_spender_summary.head(5)
top5_spender_summary = top5_spender_summary.rename(columns={"Price":"Total Purchase Value",
                                                            "Age":"Purchase Count",
                                                            "Price2":"Average Purchase Price"})
top5_spender_summary["Total Purchase Value"] = top5_spender_summary["Total Purchase Value"].map("${:.2f}".format)
top5_spender_summary["Average Purchase Price"] = top5_spender_summary["Average Purchase Price"].map("${:.2f}".format)

top5_spender_summary



,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [11]:
#Find Top5 Most Purchased Items' Purchase Count, Item Price, and Total Purchase Value and Create DataFrame Summary
popular_item_summary = purchase_data[["Item Name","Item ID" ,"SN", "Price", "Price2"]].groupby(["Item ID","Item Name"]).agg({
    "SN": "count",
    "Price": "sum",
    "Price2":"median"
})
popular_item_summary = popular_item_summary.sort_values("SN", ascending=False)
popular_item_summary = popular_item_summary.head(5)
popular_item_summary = popular_item_summary.rename(columns={"Price":"Total Purchase Value",
                                                            "SN":"Purchase Count",
                                                            "Price2":"Average Purchase Price"})
popular_item_summary["Total Purchase Value"] = popular_item_summary["Total Purchase Value"].map("${:.2f}".format)
popular_item_summary["Average Purchase Price"] = popular_item_summary["Average Purchase Price"].map("${:.2f}".format)
popular_item_summary


,,Purchase Count,Total Purchase Value,Average Purchase Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
82,Nirvana,9,$44.10,$4.90
19,"Pursuit, Cudgel of Necromancy",8,$8.16,$1.02


In [12]:
#Find Top5 Most Profitable Items' Purchase Count, Item Price, and Total Purchase Value and Create DataFrame Summary
popular_item_summary = purchase_data[["Item Name","Item ID" ,"SN", "Price", "Price2"]].groupby(["Item ID","Item Name"]).agg({
    "SN": "count",
    "Price": "sum",
    "Price2":"median"
})
profitable_item_summary = popular_item_summary.sort_values("Price", ascending=False)
profitable_item_summary = profitable_item_summary.head(5)
profitable_item_summary = profitable_item_summary.rename(columns={"Price":"Total Purchase Value",
                                                            "SN":"Purchase Count",
                                                            "Price2":"Average Purchase Price"})
profitable_item_summary["Total Purchase Value"] = profitable_item_summary["Total Purchase Value"].map("${:.2f}".format)
profitable_item_summary["Average Purchase Price"] = profitable_item_summary["Average Purchase Price"].map("${:.2f}".format)
profitable_item_summary

,,Purchase Count,Total Purchase Value,Average Purchase Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
92,Final Critic,8,$39.04,$4.88
103,Singed Scalpel,8,$34.80,$4.35
